# Импорты

In [1]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

output_notebook()

from utils.funcs import compute_metrics, critical_region_two_sided
from utils.plots import plot_two_coin_distributions


Loading BokehJS ...

# Расчёты

In [2]:
n = 50
p0 = 0.5
p1 = 0.6
alpha = 0.05
k_obs = 32

metrics = compute_metrics(n=n, p0=p0, p1=p1, alpha=alpha, k_obs=k_obs)
metrics


{'alpha': 0.05,
 'beta': 0.7629431215000944,
 'power': 0.23705687849990564,
 'p_value': 0.06490864707227217}

### Визуализации (в терминах монетки)

Мы подбрасываем *неизвестную* монету **n** раз и наблюдаем **k** орлов.

- **H0 (честная монета)**: монета честная, значит \(p = 0.5\). При H0 имеем \(k \sim \text{Binomial}(n, 0.5)\).
- **H1 (нечестная монета)**: монета смещённая, значит \(p = p_1\) (например 0.6). При H1 имеем \(k \sim \text{Binomial}(n, p_1)\).

На графике показаны *два дискретных распределения* (PMF):
- **Синее (H0)**: сколько орлов мы ожидали бы увидеть, если монета честная.
- **Оранжевое (H1)**: сколько орлов мы ожидали бы увидеть, если монета смещённая.

#### Как мы принимаем решение
Мы выбираем уровень значимости **alpha** и определяем **критическую область** при H0.
- **Красные точки** — исходы, при которых мы **отвергаем H0** ("монета не честная").
- Если монета на самом деле честная (H0 истинна), то отвергнуть H0 — это **ошибка I рода**. Её вероятность примерно равна **alpha**.

Если монета на самом деле смещённая (H1 истинна), мы всё равно можем не заметить смещение:
- **Зелёные точки** — исходы, при которых мы **не отвергаем H0**, хотя H1 истинна.
- Это вероятность **beta** (ошибка II рода).
- **Мощность теста = 1 - beta**.

#### Что означает p-value на графике
Для наблюдаемого **k** (вертикальная чёрная линия) мы считаем **двустороннее p-value** при H0.
- **Фиолетовые точки** — "хвосты" p-value: исходы, которые при H0 "не менее экстремальны", чем наблюдение.
- Правило решения по p-value: **отвергаем H0, если p-value <= порог**.

Обычно порог берут равным **alpha**, но можно менять их независимо и смотреть, как меняется решение.

In [3]:
rng = np.random.default_rng(42)

n = 50
p0 = 0.5
p1 = 0.6
alpha = 0.05
p_value_threshold = 0.05

k_obs = int(rng.binomial(n=n, p=p1))
k_obs


27

### Шаг 1. Что мы задали и что наблюдаем

- **n** — число бросков.
- **k_obs** — число орлов, которое мы *фактически увидели*.
- **p0** — вероятность орла по H0 (честная монета).
- **p1** — вероятность орла по H1 (пример "нечестной" монеты).

В этом ноутбуке `k_obs` мы иногда **симулируем** как результат бросков нечестной монеты (p=p1), чтобы наглядно увидеть, как тест себя ведёт. Если у тебя есть реальное наблюдение — просто задай `k_obs` вручную.

In [4]:
metrics = compute_metrics(n=n, p0=p0, p1=p1, alpha=alpha, k_obs=k_obs)

lower, upper = critical_region_two_sided(n=n, p0=p0, alpha=alpha)
reject_by_alpha_region = (k_obs <= lower) or (k_obs >= upper)
reject_by_p_value = metrics["p_value"] <= p_value_threshold

{
    "n": n,
    "k_obs": k_obs,
    "H0_p": p0,
    "H1_p": p1,
    "alpha": metrics["alpha"],
    "beta": metrics["beta"],
    "power": metrics["power"],
    "p_value": metrics["p_value"],
    "critical_region_two_sided": (lower, upper),
    "decision_by_alpha_region": "reject H0" if reject_by_alpha_region else "keep H0",
    "decision_by_p_value": "reject H0" if reject_by_p_value else "keep H0",
}


{'n': 50,
 'k_obs': 27,
 'H0_p': 0.5,
 'H1_p': 0.6,
 'alpha': 0.05,
 'beta': 0.7629431215000944,
 'power': 0.23705687849990564,
 'p_value': 0.6718110337653656,
 'critical_region_two_sided': (17, 33),
 'decision_by_alpha_region': 'keep H0',
 'decision_by_p_value': 'keep H0'}

### Шаг 2. Как читаем метрики и вывод

- **p_value** — насколько наше наблюдение `k_obs` "экстремально" при условии, что H0 (монета честная) верна.
- **decision_by_p_value** — решение по правилу: отвергаем H0, если `p_value <= p_value_threshold`.

Параллельно есть эквивалентное (для фиксированного alpha) правило через **критическую область**:
- `critical_region_two_sided = (lower, upper)`
- **decision_by_alpha_region** — отвергаем H0, если `k_obs <= lower` или `k_obs >= upper`.

Почему бывают два решения?
- Обычно берут `p_value_threshold = alpha`, и тогда решения **совпадают**.
- Если задать порог p-value отдельно от alpha, можно специально посмотреть, как меняется поведение теста.

In [5]:
plot = plot_two_coin_distributions(
    n=n,
    p0=p0,
    p1=p1,
    alpha=alpha,
    k_obs=k_obs,
    p_value_threshold=p_value_threshold,
    title=f"Одно наблюдение: n={n}, k={k_obs}; alpha={alpha}",
)
show(plot)


### Шаг 3. Как влияет число бросков n

Теперь фиксируем H0/H1 (p0 и p1) и смотрим, что происходит при разных `n`.

Интуиция:
- при большем `n` распределения H0 и H1 **разъезжаются** относительно друг друга,
- область beta обычно **уменьшается**, а мощность **растёт** (становится легче отличить честную монету от смещённой).

In [6]:
p0 = 0.5
p1 = 0.6
alpha = 0.05
p_value_threshold = 0.05

plots_by_n = []
for n in [20, 50, 100]:
    k_obs = int(rng.binomial(n=n, p=p1))
    plots_by_n.append(
        plot_two_coin_distributions(
            n=n,
            p0=p0,
            p1=p1,
            alpha=alpha,
            k_obs=k_obs,
            p_value_threshold=p_value_threshold,
            title=f"n={n}, k={k_obs}; alpha={alpha}",
        )
    )

show(column(*plots_by_n))


### Шаг 4. Как влияет alpha

Теперь фиксируем `n`, `p0`, `p1` и одно и то же наблюдение `k_obs`, но меняем **alpha**.

Интуиция:
- больше alpha → критическая область шире → легче отвергнуть H0 → меньше beta (выше power), но больше риск ошибки I рода;
- меньше alpha → критическая область уже → сложнее отвергнуть H0 → больше beta (ниже power), но меньше риск ошибки I рода.

In [7]:
n = 50
p0 = 0.5
p1 = 0.6
k_obs = 30

alpha_compare = row(
    plot_two_coin_distributions(
        n=n,
        p0=p0,
        p1=p1,
        alpha=0.10,
        k_obs=k_obs,
        p_value_threshold=0.10,
        title=f"n={n}, k={k_obs}; alpha=0.10",
    ),
    plot_two_coin_distributions(
        n=n,
        p0=p0,
        p1=p1,
        alpha=0.01,
        k_obs=k_obs,
        p_value_threshold=0.01,
        title=f"n={n}, k={k_obs}; alpha=0.01",
    ),
)

show(alpha_compare)
